## imports

In [1]:
!pip install datasets transformers huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 104.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.5 MB/s eta 0:00:00


In [2]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 8.2 MB/s eta 0:00:00


In [3]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [4]:
!pip install wandb  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.9 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=28119ff04f9b18c6358cd38aebfc7bbaa3e2fca3af48186db116ef3ef082897a
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [5]:
import torch
import numpy as np
import pandas as pd

from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer

from datasets import load_dataset, load_metric


from google.colab import drive
import wandb
from huggingface_hub import notebook_login

import evaluate

from sklearn.metrics import accuracy_score, f1_score

In [6]:
torch.cuda.is_available()

True

## login

In [7]:
# notebook_login()

In [8]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [9]:
drive.mount('/content/drive')

Mounted at /content/drive


## dataset

In [10]:
from datasets import interleave_datasets, Features, Value, ClassLabel

In [11]:
test_path = '/content/drive/My Drive/datasets/IMDB/IMDB_test.csv'

In [12]:
train_path_1 = '/content/drive/My Drive/datasets/IMDB/IMDB_train_1.csv'
train_path_2 = '/content/drive/My Drive/datasets/IMDB/IMDB_train_2.csv'

In [ ]:
imdb1 = load_dataset("csv", data_files={"train": train_path_1, "test": test_path})
imdb2 = load_dataset("csv", data_files={"train": train_path_2, "test": test_path})

In [18]:
imdb1, imdb2 = imdb1.remove_columns(["binary_label"]), imdb2.remove_columns(["binary_label"]),
imdb1, imdb2 = imdb1.rename_column("star_label", "label"), imdb2.rename_column("star_label", "label")
imdb1, imdb2 = imdb1.rename_column("review_text", "text"), imdb2.rename_column("review_text", "text")

In [19]:
train1, train2 = imdb1["train"].shuffle(seed=41), imdb2["train"].shuffle(seed=41)
val_test_dataset = imdb1["test"].shuffle(seed=41).select(range(1000))

In [20]:
train_dataset = interleave_datasets([train1, train2], stopping_strategy="all_exhausted")
val_dataset = val_test_dataset.select(range(500))
test_dataset = val_test_dataset.select(range(500, 1000))

In [21]:
print(train_dataset[0])
print(val_dataset[0])
print(test_dataset[0])

{'text': "What is supposed to be a simple generic mystery plot involving a dead philanthropist is, in fact, a head-ache inducing tale about a bunch of characters (the only big actor being Ginger Rogers, in a very early role) all trying to find the murderer among a small cast of residents in a posh apartment building. These characters range from utterly stupid to downright mean. As a cheap, low budget production, most of the action revolves around Rogers and her lead man (some guy, I don't care who he is 'cause he really sucked) talking about their various possibilities of solving the crime, while being constantly cut off by an absurd detective with his head in his butt. Honestly, I've never had a worse time watching an old b-rate movie of this type, and I've seen some real head-slappers.<br /><br />Oh, and the butler didn't do it, because there wasn't a butler. But pay attention to the guy who's closest to a butler. There ya go.<br /><br />--PolarisDiB", 'label': 0}
{'text': 'Back in t

## tokenizer

In [22]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") # distilbert-base-uncased

In [23]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

In [24]:
encoded_train = train_dataset.map(preprocess_function, batched=True, batch_size=None)
encoded_val = val_dataset.map(preprocess_function, batched=True, batch_size=None)
encoded_test = test_dataset.map(preprocess_function, batched=True, batch_size=None)

Map:   0%|          | 0/40534 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [25]:
encoded_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])
encoded_val.set_format("torch", columns=["input_ids", "attention_mask", "label"])
encoded_test.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [26]:
encoded_train.features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

## model

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [28]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=10).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [29]:
# del model

In [30]:
# torch.cuda.empty_cache()

In [31]:
# import gc
# gc.collect()

In [32]:
# torch.cuda.empty_cache()

## metrics

In [33]:
def binary_search(arr):
    pn = np.argmax([np.sum(arr[:4]), np.sum(arr[6:])])
    if pn == 0:
        pn = np.argmax([np.sum(arr[:2]), np.sum(arr[2:4])])
        if pn == 0:
            return np.argmax([arr[0], arr[1]]) + 0
        else:
            return np.argmax([arr[2], arr[3]]) + 2
    else:
        pn = np.argmax([np.sum(arr[6:8]), np.sum(arr[8:10])])
        if pn == 0:
            return np.argmax([arr[6], arr[7]]) + 6
        else:
            return np.argmax([arr[8], arr[9]]) + 8
      

In [34]:
def pos_neg_search(arr):
    return np.argmax([np.sum(arr[:4]), np.sum(arr[6:])])

In [35]:
def get_binary_label(x):
    if x <= 3:
        return 0
    elif x >= 6:
        return 1
    return 2

In [36]:
def compute_metrics(pred):
    labels_10 = pred.label_ids
    labels_3 = np.vectorize(get_binary_label)(labels_10)

    pred_3 = np.apply_along_axis(pos_neg_search, 1, pred.predictions)
    pred_10 = np.apply_along_axis(binary_search, 1, pred.predictions)

    f1_10 = f1_score(labels_10, pred_10, average="weighted")
    acc_10 = accuracy_score(labels_10, pred_10)
    f1_3 = f1_score(labels_3, pred_3, average="weighted")
    acc_3 = accuracy_score(labels_3, pred_3)

    return {"accuracy_10": acc_10, 
            "f1_10": f1_10, 
            "accuracy_3": acc_3, 
            "f1_3": f1_3,
            }

## trainer

In [37]:
wandb.init(project="bert_sentiment")

wandb: Currently logged in as: c-nemo. Use `wandb login --relogin` to force relogin


In [38]:
300 // 15 // 10

2

In [39]:
batch_size = 16
steps = 50 # 50 | len(encoded_train) // batch_size // ?

training_args = TrainingArguments(
    
    output_dir='/content/drive/My Drive/models/rc_binary_logs_5',
    report_to="wandb",

    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    num_train_epochs=5, # 8

    evaluation_strategy="steps", # "epoch"
    eval_steps=steps,

    logging_steps=steps,

    save_strategy="steps", # steps,
    save_steps=steps,
    save_total_limit=1,

    push_to_hub=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1_10"
)

trainer = Trainer(
    model=model, args=training_args,
    train_dataset=encoded_train, eval_dataset=encoded_val,
    compute_metrics=compute_metrics
)

In [40]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy 10,F1 10,Accuracy 3,F1 3
50,2.176500,2.093690,0.154000,0.111536,0.622000,0.569207
100,1.981700,1.736449,0.376000,0.334468,0.898000,0.897915
150,1.726100,1.570931,0.388000,0.360825,0.908000,0.908018
200,1.684200,1.527430,0.372000,0.344777,0.906000,0.905868
250,1.592200,1.449495,0.438000,0.387450,0.916000,0.915771
300,1.567800,1.407950,0.444000,0.385711,0.930000,0.929901
350,1.543000,1.376452,0.446000,0.390911,0.928000,0.928000
400,1.502900,1.377956,0.442000,0.395897,0.924000,0.923946
450,1.537300,1.398284,0.376000,0.394710,0.938000,0.938008
500,1.485500,1.398726,0.438000,0.395974,0.914000,0.913747


KeyboardInterrupt: ignored

In [41]:
results = trainer.evaluate()
results

Step,Training Loss,Validation Loss,Accuracy 10,F1 10,Accuracy 3,F1 3
50,2.176500,2.093690,0.154000,0.111536,0.622000,0.569207
100,1.981700,1.736449,0.376000,0.334468,0.898000,0.897915
150,1.726100,1.570931,0.388000,0.360825,0.908000,0.908018
200,1.684200,1.527430,0.372000,0.344777,0.906000,0.905868
250,1.592200,1.449495,0.438000,0.387450,0.916000,0.915771
300,1.567800,1.407950,0.444000,0.385711,0.930000,0.929901
350,1.543000,1.376452,0.446000,0.390911,0.928000,0.928000
400,1.502900,1.377956,0.442000,0.395897,0.924000,0.923946
450,1.537300,1.398284,0.376000,0.394710,0.938000,0.938008
500,1.485500,1.398726,0.438000,0.395974,0.914000,0.913747


{'eval_loss': 1.422710657119751,
 'eval_accuracy_10': 0.532,
 'eval_f1_10': 0.5432667176063869,
 'eval_accuracy_3': 0.958,
 'eval_f1_3': 0.9580082336137883}

In [42]:
wandb.finish()

eval/accuracy_10,▁▅▆▅▆▆▇▇▆▇▆▆▇▆▇▇▇▆▇▇▇▆▇▇██▆▇▇█▇▇███▇█▇▇█
eval/accuracy_3,▁▇▇▇██▇█████████████████████████████████
eval/f1_10,▁▅▅▆▆▆▆▇▇▇▇▆█▆▇█▇▆▇██▇████▇███▇▇███▇█▇██
eval/f1_3,▁▇▇███▇█████████████████████████████████
eval/loss,█▄▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▂▂▂▁▁▂▂▁▁▂▂▂▁▂▂▂▂▂▃
eval/runtime,▄▂▂▁▃▂▂▂▂▂▁▂▂▂▂▂▁▂▃▂▃▃▂▂▂▁▃▂▂▁▂▂▂▂▁▂▃▂▂█
eval/samples_per_second,▅▇▇█▆▇▇▇▇▇█▇▇▇▇▇█▇▆▇▆▆▇▇▇█▆▇▇▇▇▇▇▇█▇▆▇▇▁
eval/steps_per_second,▅▇▇█▆▇▇▇▇▇█▇▇▇▇▇█▇▆▇▆▆▇▇▇█▆▇▇▇▇▇▇▇▇▇▆▇▇▁
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁


In [43]:
preds_output = trainer.predict(encoded_test)
preds_output.metrics

Step,Training Loss,Validation Loss,Accuracy 10,F1 10,Accuracy 3,F1 3
50,2.176500,2.093690,0.154000,0.111536,0.622000,0.569207
100,1.981700,1.736449,0.376000,0.334468,0.898000,0.897915
150,1.726100,1.570931,0.388000,0.360825,0.908000,0.908018
200,1.684200,1.527430,0.372000,0.344777,0.906000,0.905868
250,1.592200,1.449495,0.438000,0.387450,0.916000,0.915771
300,1.567800,1.407950,0.444000,0.385711,0.930000,0.929901
350,1.543000,1.376452,0.446000,0.390911,0.928000,0.928000
400,1.502900,1.377956,0.442000,0.395897,0.924000,0.923946
450,1.537300,1.398284,0.376000,0.394710,0.938000,0.938008
500,1.485500,1.398726,0.438000,0.395974,0.914000,0.913747


{'test_loss': 1.4217084646224976,
 'test_accuracy_10': 0.53,
 'test_f1_10': 0.5393982348894806,
 'test_accuracy_3': 0.94,
 'test_f1_3': 0.9399980798771123,
 'test_runtime': 18.1691,
 'test_samples_per_second': 27.519,
 'test_steps_per_second': 1.761}

## save

In [44]:
trainer.save_model('/content/drive/My Drive/models/bert-binary-5/')

In [45]:
model.save_pretrained('/content/drive/My Drive/models/reserve/bert-binary-5/model/')
tokenizer.save_pretrained('/content/drive/My Drive/models/reserve/bert-binary-5/tokenizer/')

('/content/drive/My Drive/models/reserve/bert-binary-5/tokenizer/tokenizer_config.json',
 '/content/drive/My Drive/models/reserve/bert-binary-5/tokenizer/special_tokens_map.json',
 '/content/drive/My Drive/models/reserve/bert-binary-5/tokenizer/vocab.txt',
 '/content/drive/My Drive/models/reserve/bert-binary-5/tokenizer/added_tokens.json',
 '/content/drive/My Drive/models/reserve/bert-binary-5/tokenizer/tokenizer.json')

In [51]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [56]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.cache/huggingface

In [58]:
model.push_to_hub('c-nemo/bert-for-movie-review-classification')

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

HfHubHTTPError: ignored

In [ ]:
preds_output

In [49]:
preds_output.label_ids

array([6, 7, 6, 9, 7, 1, 2, 1, 9, 0, 0, 3, 8, 1, 2, 6, 7, 9, 6, 1, 0, 0,
       2, 3, 8, 6, 1, 8, 6, 7, 8, 9, 9, 9, 9, 3, 2, 2, 3, 3, 3, 7, 6, 0,
       2, 0, 7, 7, 9, 0, 3, 6, 0, 0, 0, 1, 0, 8, 9, 3, 0, 3, 2, 6, 9, 2,
       9, 2, 8, 0, 7, 2, 9, 7, 2, 6, 0, 3, 9, 9, 0, 8, 2, 9, 3, 7, 7, 9,
       9, 6, 8, 6, 3, 3, 0, 7, 7, 9, 0, 6, 0, 0, 7, 6, 6, 0, 9, 9, 6, 9,
       7, 8, 0, 7, 9, 0, 8, 9, 9, 8, 6, 2, 0, 9, 2, 9, 2, 1, 0, 9, 0, 7,
       2, 1, 9, 0, 2, 0, 2, 8, 0, 0, 7, 7, 3, 9, 7, 8, 1, 9, 8, 0, 3, 9,
       3, 3, 3, 9, 3, 9, 0, 9, 9, 0, 7, 9, 0, 7, 9, 2, 3, 3, 1, 9, 1, 8,
       0, 2, 9, 3, 9, 3, 0, 0, 3, 0, 0, 6, 7, 6, 2, 3, 1, 9, 2, 0, 2, 1,
       2, 6, 0, 1, 3, 9, 6, 0, 9, 9, 3, 3, 9, 3, 1, 3, 9, 0, 0, 2, 9, 7,
       3, 2, 6, 8, 3, 1, 0, 7, 8, 6, 3, 2, 2, 2, 6, 0, 0, 7, 7, 7, 9, 2,
       2, 9, 9, 1, 7, 9, 0, 8, 9, 0, 9, 8, 9, 8, 6, 0, 1, 1, 0, 0, 0, 6,
       8, 2, 2, 0, 7, 0, 0, 7, 6, 9, 7, 2, 6, 6, 9, 8, 0, 9, 3, 2, 8, 3,
       9, 6, 0, 8, 9, 3, 9, 2, 2, 0, 0, 2, 6, 9, 6,

In [47]:
labels_10 = preds_output.label_ids
labels_3 = np.vectorize(get_binary_label)(labels_10)

pred_3 = np.apply_along_axis(pos_neg_search, 1, preds_output.predictions)
pred_10 = np.apply_along_axis(binary_search, 1, preds_output.predictions)

In [ ]:
# true_10 = np.array(encoded_test["star_label"])
# true_3= np.array(encoded_test["binary_label"])

In [48]:
new_df = pd.DataFrame( )
new_df['true_10'] = labels_10
new_df['true_3'] = labels_3
new_df['pred_10'] = pred_10
new_df['pred_3'] = pred_3
new_df

,true_10,true_3,pred_10,pred_3
0,6,1,6,1
1,7,1,2,0
2,6,1,6,1
3,9,1,8,1
4,7,1,9,1
...,...,...,...,...
495,3,0,9,1
496,9,1,9,1
497,2,0,7,1
498,6,1,6,1


In [50]:
new_df.to_csv('/content/drive/MyDrive/preds_binary-5.csv', header=True, index=False)